In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [26]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read in CSV and Perform Data Cleaning

In [27]:
# Load the data
df = pd.read_csv('Resources/LoanStats_2019Q1.csv')

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500,0.1719,375.35,RENT,66000.0,Source Verified,Mar-19,low_risk,n,27.24,...,85.7,100.0,0,0,65687,38199,2000,61987,N,N
1,25000,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-19,low_risk,n,20.23,...,91.2,50.0,1,0,271427,60641,41200,49197,N,N
2,20000,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-19,low_risk,n,24.26,...,66.7,50.0,0,0,60644,45684,7500,43144,N,N
3,10000,0.1640,353.55,RENT,92000.0,Verified,Mar-19,low_risk,n,31.44,...,100.0,50.0,1,0,99506,68784,19700,76506,N,N
4,22000,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-19,low_risk,n,18.76,...,100.0,0.0,0,0,219750,25919,27600,20000,N,N


## Split Data into Training and Testing

In [28]:
# Create new dataframe out of get_dummies function
df = pd.get_dummies(df)
df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-19,next_pymnt_d_May-19,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500,0.1719,375.35,66000.0,27.24,0,0,8,0,1609,...,1,1,0,1,0,1,1,0,1,1
1,25000,0.2000,929.09,105000.0,20.23,0,0,17,1,18368,...,1,1,0,1,0,1,1,0,1,1
2,20000,0.2000,529.88,56000.0,24.26,0,0,8,0,13247,...,1,1,0,1,0,1,1,0,1,1
3,10000,0.1640,353.55,92000.0,31.44,0,1,10,1,17996,...,1,1,0,1,0,1,1,0,1,1
4,22000,0.1474,520.39,52000.0,18.76,0,1,14,0,9091,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000,0.1502,346.76,26000.0,9.60,0,0,9,0,2684,...,1,1,0,1,0,1,1,0,1,1
68813,12000,0.2727,368.37,63000.0,29.07,0,0,8,0,13314,...,1,1,0,1,0,1,1,0,1,1
68814,5000,0.1992,185.62,52000.0,14.86,0,0,5,1,3715,...,1,1,0,1,0,1,1,0,1,1
68815,40000,0.0646,1225.24,520000.0,9.96,0,1,21,0,59529,...,1,1,1,0,0,1,1,0,1,1


In [29]:
X = df.drop(columns = 'loan_status_low_risk')

In [30]:
# Create Target
y = df['loan_status_low_risk']

In [31]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_high_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-19,next_pymnt_d_May-19,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.005042,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.070831,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [32]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status_low_risk, dtype: int64

In [33]:
# split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 96)

## Data Processing

In [34]:
# Create the StandardScaler instance
from sklearn.preprocessing import LabelEncoder, StandardScaler

data_scaler = StandardScaler()

In [35]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X_train)

StandardScaler()

In [36]:
# Scale the training and testing data
X_train_scaled = data_scaler.transform(X_train)
X_test_scaled = data_scaler.transform(X_test)

# Ensemble Learners

## Balanced Random Forest Classifier

In [37]:
 # Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brf_model = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
brf_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [38]:
# Calculated the balanced accuracy score
y_pred = brf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9989776843089146

In [39]:
# Display the confusion matrix
y_pred = brf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   87,     0],
       [   35, 17083]], dtype=int64)

In [17]:
 # Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.67      0.91      0.07      0.78      0.59        87
   low_risk       1.00      0.91      0.67      0.95      0.78      0.62     17118

avg / total       0.99      0.91      0.67      0.95      0.78      0.62     17205



In [40]:
 # List the features sorted in descending order by feature importance
importances_sorted = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.4319074770416764, 'loan_status_high_risk'),
 (0.048217737882456556, 'total_rec_prncp'),
 (0.04789904198551086, 'last_pymnt_amnt'),
 (0.0440165962029668, 'total_rec_int'),
 (0.04151201809415954, 'total_pymnt'),
 (0.04115171848458576, 'total_pymnt_inv'),
 (0.012016253406858933, 'int_rate'),
 (0.01086692305958722, 'mths_since_recent_inq'),
 (0.0102872715125112, 'out_prncp_inv'),
 (0.010249171606419819, 'issue_d_Jan-19')]

## Easy Ensemble Classifier

In [42]:
 # Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

eec_model = EasyEnsembleClassifier(n_estimators = 100, random_state = 1)
eec_model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [43]:
# Calculated the balanced accuracy score
y_pred = eec_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [44]:
# Display the confusion matrix
y_pred = eec_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   87,     0],
       [    0, 17118]], dtype=int64)

In [45]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00        87
          1       1.00      1.00      1.00      1.00      1.00      1.00     17118

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205

